In [14]:
import os
import time
import json
import pandas as pd
from tqdm.notebook import tqdm

import lfqa_utils as utils

In [11]:
standard_columns = ['system', 'dataset', 'language', 'id', 'experiment_url']

In [23]:
system = 'deeppavlov'
dataset = 'qald9plustrain'
language = 'ru'
kg = 'wikidata'

data_files = [file for file in os.listdir("../translated_data/") if system in file.lower() and dataset.replace('-', '') in file.lower() and f"{language}-" in file.lower() and ".log" not in file.lower() and kg in file.lower()]
len(data_files)

18

In [24]:
for data_file in tqdm(data_files):
    data = utils.read_json(f"../translated_data/{data_file}")

    if not data:
        print(f"Error with {data_file}")
        assert False
    else:
        data = data['questions']
        
    df_dict = {column: [] for column in standard_columns + ['precision', 'recall', 'f1']} 

    for question in data:
        id_ = question["id"]

        answers_pred = question["answers_pred"]
        if type(answers_pred) == dict:
            answers_pred_formatted = [{"x": {"type": "uri", "value": value}} for value in answers_pred['answer']]
        else:
            answers_pred_formatted = []

        answers_true_formatted = question['answers'][0]['results']['bindings'] if "results" in question['answers'][0] else []
        
        precision, recall, f1 = utils.precision_recall_f1(answers_true_formatted, answers_pred_formatted, utils.is_equal)

        experiment_url = "http://custom-f1-calc.here"

        df_dict['system'].append(system)
        df_dict['dataset'].append(dataset)
        df_dict['language'].append(language)
        df_dict['id'].append(id_)
        df_dict['experiment_url'].append(experiment_url)
        df_dict['precision'].append(precision)
        df_dict['recall'].append(recall)
        df_dict['f1'].append(f1)

    df = pd.DataFrame.from_dict(df_dict)
    df.to_csv(f"../translated_data/finished_experiments_custom/{data_file}.tsv", sep='\t', index=False)

  0%|          | 0/18 [00:00<?, ?it/s]